In [1]:
import numpy as np

import matplotlib.pyplot as plt

%matplotlib inline

## 2 -Gait Modeal:  LSTMs in Keras: 

 LSTM model that takes as inputs my gait angles features.


In [47]:
import numpy as np
np.random.seed(0)
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
import tensorflow as tf
np.random.seed(1)

In [131]:
# load data with Pandas
import pandas as pd
X_train = pd.read_csv("cycles_for_LSTM/X_train.csv")
Y_train = pd.read_csv("cycles_for_LSTM/Y_train.csv")
X_val = pd.read_csv("cycles_for_LSTM/X_val.csv")
Y_val = pd.read_csv("cycles_for_LSTM/Y_val.csv")
X_test = pd.read_csv("cycles_for_LSTM/X_test.csv")
Y_test = pd.read_csv("cycles_for_LSTM/Y_test.csv")
X_all = pd.read_csv("cycles_for_LSTM/X_all.csv")
Y_all = pd.read_csv("cycles_for_LSTM/Y_all.csv")
# print things about the files
print(X_train.shape)
print(Y_train.shape)
print(X_val.shape)
print(Y_val.shape)
print(X_test.shape)
print(Y_test.shape)
#print(X_train.head(1))

(232, 2162)
(232, 1)
(498, 2162)
(498, 1)
(120, 2162)
(120, 1)


In [26]:
def Reshape_to_input(X):
    """
    reshapes data
    """
    max_length = 15
    [Num_examples, dim] = X.shape
    input_X = X.values[:,2:].reshape(Num_examples, max_length, 144)
    
    return input_X
    

In [78]:
X_train = Reshape_to_input(X_train)
X_val = Reshape_to_input(X_val)
X_test = Reshape_to_input(X_test)
X_all = Reshape_to_input(X_all)

In [214]:
def convert_to_one_hot(inputY, C):
    N= inputY.size
    Y=np.zeros((N,C))
    for i in range (0, inputY.size):
        Y[i, inputY[i]-1] = 1
        
    
    return Y


In [201]:
print(Y_train.values.size)

232


In [138]:
print(X_all.shape)
#X_all[(25,5),:,:].shape
indices = [0, 1, 4]
#F = np.take(X_all, (indices))
F=X_all.as_matrix()[indices]
print(F.shape)

(498, 2162)
(3, 2162)


### 2.2 Keras and mini-batching 

In this exercise, we want to train Keras using mini-batches. However, most deep learning frameworks require that all sequences in the same mini-batch have the same length. This is what allows vectorization to work: If you had a 3-word sentence and a 4-word sentence, then the computations needed for them are different (one takes 3 steps of an LSTM, one takes 4 steps) so it's just not possible to do them both at the same time.

The common solution to this is to use padding. Specifically, set a maximum sequence length, and pad all sequences to the same length. For example, of the maximum sequence length is 20, we could pad every sentence with "0"s so that each input sentence is of length 20. Thus, a sentence "i love you" would be represented as $(e_{i}, e_{love}, e_{you}, \vec{0}, \vec{0}, \ldots, \vec{0})$. In this example, any sentences longer than 20 words would have to be truncated. One simple way to choose the maximum sequence length is to just pick the length of the longest sentence in the training set. 


In [183]:
def Gait_model(data):
    """
    Function creating the Emojify-v2 model's graph.
    
    Arguments:
    input - data

    Returns:
    model -- a model instance in Keras
    """
   
    ### START CODE HERE ###
    gait_data = Input(data.shape, dtype='float32')
    print(gait_data.shape)
    #gait_data = tf.keras.backend.squeeze(gait_data, 0)
    #examples = gait_data
    #max_length = 15
    #examples = examples[2:,:]
    #print(examples.shape)
    #Num_examples = examples.shape[0]
    #embeddings = examples.reshape(Num_examples, max_length, 144)
    #print(embeddings)
    #embeddings = X_train# Reshape_to_input(X_train)
    #embeddings = tf.convert_to_tensor(embeddings, name="raw_data", dtype=tf.float32)

    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.
    print(gait_data.shape)
    X = LSTM(128, return_sequences=True)( gait_data)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a single hidden state, not a batch of sequences.
    X = LSTM(128, return_sequences=False)(X)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
    X = Dense(3)(X)
    # Add a softmax activation
    X = Activation('softmax')(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs=gait_data, outputs=X)
    
    ### END CODE HERE ###
    
    return model

Run the following cell to create your model and check its summary. Because all sentences in the dataset are less than 10 words, we chose `max_len = 10`.  You should see your architecture, it uses "20,223,927" parameters, of which 20,000,050 (the word embeddings) are non-trainable, and the remaining 223,877 are. Because our vocabulary size has 400,001 words (with valid indices from 0 to 400,000) there are 400,001\*50 = 20,000,050 non-trainable parameters. 

In [184]:
maxLen = 15
print(X.shape)
model = Gait_model((X[0,:,:]))
model.summary()

(232, 15, 144)
(?, 15, 144)
(?, 15, 144)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_40 (InputLayer)        (None, 15, 144)           0         
_________________________________________________________________
lstm_42 (LSTM)               (None, 15, 128)           139776    
_________________________________________________________________
dropout_33 (Dropout)         (None, 15, 128)           0         
_________________________________________________________________
lstm_43 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dropout_34 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 3)                 387       
_________________________________________________________________
activation_17 (Activation)   (None,

In [158]:
X = Reshape_to_input(X_train)

As usual, after creating your model in Keras, you need to compile it and define what loss, optimizer and metrics your are want to use. Compile your model using `categorical_crossentropy` loss, `adam` optimizer and `['accuracy']` metrics:

In [185]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

It's time to train your model. Your Emojifier-V2 `model` takes as input an array of shape (`m`, `max_len`) and outputs probability vectors of shape (`m`, `number of classes`). We thus have to convert X_train (array of sentences as strings) to X_train_indices (array of sentences as list of word indices), and Y_train (labels as indices) to Y_train_oh (labels as one-hot vectors).

In [215]:
#X_train_indices = sentences_to_indices(X_train, word_to_index, maxLen)
Y_train_oh = convert_to_one_hot(Y_train.values, C = 3)

In [220]:
X_test = Reshape_to_input(X_test)
Y_test_oh = convert_to_one_hot(Y_test.values, C = 3)

In [225]:
X_val = Reshape_to_input(X_val)
Y_val_oh = convert_to_one_hot(Y_val.values, C = 3)

Fit the Keras model on `X_train_indices` and `Y_train_oh`. We will use `epochs = 50` and `batch_size = 32`.

In [243]:
model.fit(X, Y_train_oh, epochs = 25, batch_size = 20, shuffle=True)

Epoch 1/25
232/232 [==============================] - 2s 8ms/step - loss: 0.0917 - acc: 0.9353
Epoch 2/25
232/232 [==============================] - 1s 5ms/step - loss: 0.0897 - acc: 0.9569A: 0s - loss: 0.0921 - acc: 0.
Epoch 3/25
232/232 [==============================] - 1s 4ms/step - loss: 0.0900 - acc: 0.9569
Epoch 4/25
232/232 [==============================] - 1s 3ms/step - loss: 0.0922 - acc: 0.9526
Epoch 5/25
232/232 [==============================] - 1s 2ms/step - loss: 0.0925 - acc: 0.9353
Epoch 6/25
232/232 [==============================] - 1s 2ms/step - loss: 0.0925 - acc: 0.9397
Epoch 7/25
232/232 [==============================] - 1s 2ms/step - loss: 0.0938 - acc: 0.9397
Epoch 8/25
232/232 [==============================] - 1s 2ms/step - loss: 0.0932 - acc: 0.9397
Epoch 9/25
232/232 [==============================] - 1s 2ms/step - loss: 0.0893 - acc: 0.9655
Epoch 10/25
232/232 [==============================] - 1s 2ms/step - loss: 0.0948 - acc: 0.9440
Epoch 11/25
232/232

Your model should perform close to **100% accuracy** on the training set. The exact accuracy you get may be a little different. Run the following cell to evaluate your model on the test set. 

In [244]:
#X_test_indices = sentences_to_indices(X_test, word_to_index, max_len = maxLen)
#Y_test_oh = convert_to_one_hot(Y_test, C = 5)
loss, acc = model.evaluate(X_test, Y_test_oh)
print()
print("Test accuracy = ", acc)

120/120 [==============================] - 0s 2ms/step

Test accuracy =  0.616666662693


You should get a test accuracy between 80% and 95%. Run the cell below to see the mislabelled examples. 